In [ ]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Barra de progreso de un proceso
# ------------------------------------------------------------------------------
from tqdm import tqdm

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('once')

In [ ]:
# Para ejecutar en google colab
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# Para conectar con Drive
%cd /content/drive/My Drive/Colab Notebooks/

/content/drive/My Drive/Colab Notebooks


In [ ]:
import os

In [ ]:
df = pd.read_csv("project-da-promo-C-module-3-team-5/datos/4.reg_encoding_oh_final.csv", index_col=0)
df.head()

,yr,mnth,dia_anual,holiday_num,weekday_num,workingday_num,weathersit,atemp,casual,seasons_autumn,seasons_spring,seasons_summer,seasons_winter
0,0,1,1,1,0,0,2,18.18125,331,0.0,0.0,0.0,1.0
1,0,1,2,0,1,1,2,17.68695,131,0.0,0.0,0.0,1.0
2,0,1,3,0,2,1,1,9.47025,120,0.0,0.0,0.0,1.0
3,0,1,4,0,3,1,1,10.60610,108,0.0,0.0,0.0,1.0
4,0,1,5,0,4,1,1,11.46350,82,0.0,0.0,0.0,1.0


In [ ]:

X = df.drop("casual", axis = 1)
y = df["casual"]

In [ ]:
# Generamos los conjuntos de train y test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 43)

In [ ]:
# Definimos un diccionario con los hiperparámetros que deberemos ocupar en el random forest

param = {"max_depth": [4,6,8], 
        "max_features": [4,5,6,7],
        "min_samples_split": [5, 20, 75],
        "min_samples_leaf": [5,20,75]} 

In [ ]:
# Definimos un diccionario con los hiperparámetros que deberemos ocupar en el random forest

param = {"max_depth": [2, 3, 4], 
        "max_features": [1, 2, 3, 4],
        "min_samples_split": [10, 50, 100],
        "min_samples_leaf": [5,20,75]} 

In [ ]:
# Aplicamos el GridSearch con estimador de RandomForestRegressor
gs_rf = GridSearchCV(
            estimator=RandomForestRegressor(), # tipo de modelo que queremos hacer
            param_grid= param, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") # la métrica que queremos que nos devuelva

In [ ]:
# Entrenamos el modelo con .fit()
gs_rf.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [4, 6, 8], 'max_features': [4, 5, 6, 7],
                         'min_samples_leaf': [5, 20, 75],
                         'min_samples_split': [5, 20, 75]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [ ]:
# método para que elija el mejor estimador o combinación de valores
bosque = gs_rf.best_estimator_
bosque

RandomForestRegressor(max_depth=8, max_features=7, min_samples_leaf=5,
                      min_samples_split=5)

In [ ]:
# Aplicamos el método .predict() para calcular las métricas del mejor modelo.
y_pred_test_rf = bosque.predict(x_test)
y_pred_train_rf = bosque.predict(x_train)

In [ ]:
# Medimos las métricas del random forest
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [ ]:
# sacamos las métricas para ver si hay overfitting o unerfitting, para modificar la profundidad en función de estos resultados

dt_results = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "Cas_Cod_Random Forest")
dt_results

,MAE,MSE,RMSE,R2,set,modelo
0,229.217086,89910.015190,299.849988,0.823528,test,Reg_Cod_Random Forest
1,169.601716,72756.527452,269.734179,0.842137,train,Reg_Cod_Random Forest


In [ ]:
# Unimos estos resultados con los obtenidos en los modelos anteriores

df_RF_DT_results = pd.read_csv("project-da-promo-C-module-3-team-5/datos/6.cas_resultados_2_cod_est.csv", index_col = 0)
df_RF_DT_results

,MAE,MSE,RMSE,R2,set,modelo
2,526.958790,486331.406888,697.374653,0.774698,test,Reg_est_Decision tree II Est-Cod
3,538.241508,508942.300858,713.401921,0.795858,train,Reg_est_Decision tree II Est-Cod
6,403.047749,264899.404086,514.683790,0.877280,test,Reg_Cod_Decision tree II
7,387.159792,244561.022740,494.531114,0.901904,train,Reg_Cod_Decision tree II


In [ ]:
#Unimos los resultados del Random Forest y el Decision Tree:

df_ran_dec = pd.concat([df_RF_DT_results, dt_results], axis=0)
df_ran_dec

,MAE,MSE,RMSE,R2,set,modelo
2,526.958790,486331.406888,697.374653,0.774698,test,Reg_est_Decision tree II Est-Cod
3,538.241508,508942.300858,713.401921,0.795858,train,Reg_est_Decision tree II Est-Cod
6,403.047749,264899.404086,514.683790,0.877280,test,Reg_Cod_Decision tree II
7,387.159792,244561.022740,494.531114,0.901904,train,Reg_Cod_Decision tree II
0,229.217086,89910.015190,299.849988,0.823528,test,Reg_Cod_Random Forest
1,169.601716,72756.527452,269.734179,0.842137,train,Reg_Cod_Random Forest


Cotejamos todas las opciones. Tras deliberar y valorar las métricas, optamos por utilizar el modelo de Random Forest por sus puntuaciones tan similares en el R2 y el RMSE, evitando así un modelo con over o underfitting.

De esta manera, con el modelo de Random Forest nuestras variables predictoras o independientes explican un 82% de la varianza en la media de las puntuaciones de la variable resultado o dependiente. Además, cuenta con el RMSE más bajo de todos los modelos, por lo que es el modelo que más se parece a los datos originales.
